In [1]:
import certifi
import time
import sys
from datetime import timedelta, date
import datetime
import os

from variables import COUNTRIES, MONGO_URL
import json
import threading
import numpy
from sportmonks import SportMonksAPI
from pymongo import MongoClient
import pandas as pd
import boto3
from fixture_formatter import FixtureFormatter
s3 = boto3.client('s3')
bucket = "footyamigo-fixtures"

formatter = FixtureFormatter()

ca = certifi.where()

client = MongoClient(MONGO_URL, tlsCAFile=ca)
database = client.footyamigo
fixtures_col = database.fixtures
errors_collection = database.errors
users_collection = database.users


In [53]:
fixtures = fixtures_col.find({
  "result.goal_timings_all": {"$not": {"$elemMatch":{
    "$gte": 5,
    "$lte": 15
   } }}
}, {"result.goal_timings_all": 1, "result.goals_between_5_15": 1}).limit(10)

In [66]:
fixtures = fixtures_col.find({
"stats.home.goals": {"$exists": False}
    
}, {"result.goal_timings_all": 1, "result.goals_between_5_15": 1, "stats": 1}).limit(10)


In [67]:
for f in fixtures: 
  print(f)

{'_id': ObjectId('617841b53641e0b1cb529e1b'), 'result': {'goal_timings_all': [], 'goals_between_5_15': False}, 'stats': {'home': {}, 'away': {}, 'combined': {}, 'winning_team': None, 'losing_team': None, 'underdog': None, 'underdog_playing_home': None, 'favorite': None, 'favorite_playing_away': None, 'underdog_playing_away': None, 'favorite_playing_home': None}}
{'_id': ObjectId('617841ba3641e0b1cb529f38'), 'result': {'goal_timings_all': [40, 75], 'goals_between_5_15': False}, 'stats': {'home': {}, 'away': {}, 'combined': {}, 'winning_team': None, 'losing_team': None, 'underdog': None, 'underdog_playing_home': None, 'favorite': None, 'favorite_playing_away': None, 'underdog_playing_away': None, 'favorite_playing_home': None}}
{'_id': ObjectId('617841ba3641e0b1cb529f43'), 'result': {'goal_timings_all': [25], 'goals_between_5_15': False}, 'stats': {'home': {}, 'away': {}, 'combined': {}, 'winning_team': None, 'losing_team': None, 'underdog': None, 'underdog_playing_home': None, 'favorite

In [36]:
# fixture_ids = []
# for i in fixtures:
#   fixture_ids.append(i['fixture_id'])
#   print(i['fixture_id'])

In [27]:
fixtures = fixtures_col.find({
  "$ne": {"result.goal_timings_all": {"$elemMatch": {
        "$gte": 5,
        "$lte": 15
    }}
    }
}, {"result.goal_timings_all": 1, "result.goals_between_5_15": 1}).limit(10)


In [17]:
fixtures_col.update_many({
  "result.goal_timings_all": {"$elemMatch":{
    "$gte": 5,
    "$lte": 15
   } }
}, {"$set": {"result.goals_between_5_15": True, "result.goals_between_5_15_no": False}})

In [55]:
fixtures_col.update_many({
  "result.goal_timings_all": {"$not": {"$elemMatch":{
    "$gte": 5,
    "$lte": 15
   } }}
}, {"$set": {"result.goals_between_5_15": False, "result.goals_between_5_15_no": True}})

In [73]:
fixtures = fixtures_col.update_many({
  "result.o95_corners": True 
}, {"$set": {"result.u95_corners": False }})

In [74]:
fixtures = fixtures_col.update_many({
  "result.o95_corners": False 
}, {"$set": {"result.u95_corners": True }})

In [92]:
fixtures = fixtures_col.update_many({
}, [{"$set": {"home.league_position": "$home_position", "away.league_position": "$away_position" }}])

In [91]:
fixtures = fixtures_col.find({
}, {"home_position": 1, "home.league_position": 1, "away.league_position": 1, "away_position":1}).limit(10)
for f in fixtures: 
  print(f)

{'_id': ObjectId('617841b53641e0b1cb529e10'), 'away_position': 14, 'home_position': 16, 'away': {'league_position': 14}, 'home': {'league_position': 16}}
{'_id': ObjectId('617841b53641e0b1cb529e1b'), 'away_position': 1, 'home_position': 5, 'away': {}, 'home': {}}
{'_id': ObjectId('617841b63641e0b1cb529e26'), 'away_position': 10, 'home_position': 17, 'away': {}, 'home': {}}
{'_id': ObjectId('617841b63641e0b1cb529e31'), 'away_position': 3, 'home_position': 12, 'away': {}, 'home': {}}
{'_id': ObjectId('617841b63641e0b1cb529e3c'), 'away_position': 7, 'home_position': 6, 'away': {}, 'home': {}}
{'_id': ObjectId('617841b63641e0b1cb529e48'), 'away_position': 15, 'home_position': 16, 'away': {}, 'home': {}}
{'_id': ObjectId('617841b63641e0b1cb529e5c'), 'away_position': 2, 'home_position': 20, 'away': {}, 'home': {}}
{'_id': ObjectId('617841b63641e0b1cb529e67'), 'away_position': 4, 'home_position': 17, 'away': {}, 'home': {}}
{'_id': ObjectId('617841b73641e0b1cb529e72'), 'away_position': 14, 'h

In [8]:
blocked_ids = errors_collection.distinct("strategy.user_id", {"message": "A request to the Telegram API was unsuccessful. Error code: 403. Description: Forbidden: bot was blocked by the user"})

In [9]:
blocked_ids

[930, 954]

In [10]:
u = users_collection.update_many({"id": {"$in": blocked_ids}}, {"$set": {"telegram" : None}})

In [20]:

u

In [2]:
message = "'str' object has no attribute 'trim'"
message ="'home_name'"
message = "'NoneType' object is not subscriptable"

message = """error while multiplanner was selecting best plan :: caused by"""
message = "A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: chat not found"

message = "(host='api.telegram.org', port=443)"
message = "Unsupported start tag"
message = "argument must be a string or a number"
message = "'timer'"
message = "duplicate key error collection"
message = "Read-only file system"
message = "bot was blocked"

# errors_collection.delete_many({"message": {"$regex": ".*"+message+".*"}})
errors_collection.delete_many({"message":{"$regex": f".*{message}.*"}})